### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [ ]:
# !pip install streamlit
# pip install fastapi
#pip install uvicorn
#pip install "langserve[all]"

In [9]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

# import openai
# openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [10]:
# from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10ca20e20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10ca221d0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# !pip install langchain_core
# !pip install langchain-groq
# !pip install langchain

In [11]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [12]:
result

AIMessage(content='Here\'s the translation of "Hello, how are you?" in French:\n\n**Salut, comment vas-tu ?** \n\n* **Salut** is a casual way to say hello.\n* **Comment vas-tu ?** is a common way to ask someone how they are doing. \n\nLet me know if you\'d like to know other ways to say hello or ask how someone is doing!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 21, 'total_tokens': 109, 'completion_time': 0.16, 'prompt_time': 0.002253255, 'queue_time': 0.084286499, 'total_time': 0.162253255}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-be650e1d-ada4-41a4-8490-3b821541e3fa-0', usage_metadata={'input_tokens': 21, 'output_tokens': 88, 'total_tokens': 109})

In [13]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here\'s the translation of "Hello, how are you?" in French:\n\n**Salut, comment vas-tu ?** \n\n* **Salut** is a casual way to say hello.\n* **Comment vas-tu ?** is a common way to ask someone how they are doing. \n\nLet me know if you\'d like to know other ways to say hello or ask how someone is doing!\n'

In [14]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

"Hello: Salut\n\nHow are you?: Comment vas-tu ? (informal, to someone you know well) \nor\nComment allez-vous ? (formal, to someone you don't know well) \n\n\nSo, depending on the context, you could say:\n\n* **Salut, comment vas-tu ?**\n* **Salut, comment allez-vous ?**\n"

In [15]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [16]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [17]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [18]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'